In [11]:
## ref : https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/optimize-llama-2-gptq.ipynb

In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
!sudo -H pip install auto-gptq --no-cache-dir

     |████████████████████████████████| 4.8MB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 12.2MB 106.9MB/s eta 0:00:01
     |████████████████████████████████| 266kB 109.1MB/s eta 0:00:01


  Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [3]:
!sudo -H pip install --upgrade optimum

     |████████████████████████████████| 409kB 6.9MB/s eta 0:00:01


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch

In [3]:
model_name = "/data/quantization-trials/merged-model"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [2]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

In [4]:
quantizer = GPTQQuantizer(bits=4, dataset="wikitext2")
quantizer.quant_method = "gptq"

In [5]:
quantized_model = quantizer.quantize_model(model, tokenizer)

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Quantizing model.decoder.layers blocks :   0%|          | 0/24 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Found modules on cpu/disk. Using Exllama/Exllamav2 backend requires all the modules to be on GPU. Setting `disable_exllama=True`


In [6]:
quant_path = "/data/quantization-trials/GPTQ-quantized"

In [6]:
# save the quantize model to disk

quantized_model.save_pretrained(quant_path, safe_serialization=True)

### Inference on quantized model

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

In [9]:
gptq_config = GPTQConfig(bits=4, use_exllama=True)

model_id = "/data/quantization-trials/GPTQ-quantized"
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", quantization_config=gptq_config)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [10]:
def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

import time

In [11]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is science")
print("time taken is :", time.time()-start)

time taken is : 135.37412905693054


In [15]:
print(output1)

</s>what is science?

Science is the study of the world around us. It is the study of the physical, chemical, and biological world around us. It is the study of the world around us.

What is the world around us?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the physical, chemical, and biological world around us.

What is the world around us made of?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the physical, chemical, and biological world around us.

What is the world around us made of?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the physical, chemical, and biological world around us.

What is the world around us made of?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the ph

### Inference on un-quantized model

In [16]:
model_id = "/data/quantization-trials/merged-model"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [17]:
# Using unquant model
start = time.time()
output2 = predict_from_normal("what is science")
print("time taken is :", time.time()-start)

time taken is : 23.91231393814087


In [26]:
print(output2)

</s>what is science?

The word "science" is used to describe a variety of fields of study, including:

biology

chemistry

biology of the brain

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive sy